In [1]:
import psutil
import platform
from datetime import datetime

In [2]:
def get_size(bytes, suffix="B"):
    """
    Scale bytes to its proper format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

In [3]:
print("="*40, "System Information", "="*40)
uname = platform.uname()
print(f"System: {uname.system}")
print(f"Node Name: {uname.node}")
print(f"Release: {uname.release}")
print(f"Version: {uname.version}")
print(f"Machine: {uname.machine}")
print(f"Processor: {uname.processor}")

======================================== System Information ========================================
System: Windows
Node Name: DESKTOP-0MFRJ4E
Release: 10
Version: 10.0.18362
Machine: AMD64
Processor: Intel64 Family 6 Model 158 Stepping 9, GenuineIntel


In [4]:
# Boot Time
print("="*40, "Boot Time", "="*40)
boot_time_timestamp = psutil.boot_time()
bt = datetime.fromtimestamp(boot_time_timestamp)
print(f"Boot Time: {bt.year}/{bt.month}/{bt.day} {bt.hour}:{bt.minute}:{bt.second}")

======================================== Boot Time ========================================
Boot Time: 2020/3/4 8:34:26


In [5]:
# let's print CPU information
print("="*40, "CPU Info", "="*40)
# number of cores
print("Physical cores:", psutil.cpu_count(logical=False))
print("Total cores:", psutil.cpu_count(logical=True))
# CPU frequencies
cpufreq = psutil.cpu_freq()
print(f"Max Frequency: {cpufreq.max:.2f}Mhz")
print(f"Min Frequency: {cpufreq.min:.2f}Mhz")
print(f"Current Frequency: {cpufreq.current:.2f}Mhz")
# CPU usage
print("CPU Usage Per Core:")
for i, percentage in enumerate(psutil.cpu_percent(percpu=True)):
    print(f"Core {i}: {percentage}%")
print(f"Total CPU Usage: {psutil.cpu_percent()}%")

======================================== CPU Info ========================================
Physical cores: 4
Total cores: 4
Max Frequency: 3000.00Mhz
Min Frequency: 0.00Mhz
Current Frequency: 3000.00Mhz
CPU Usage Per Core:
Core 0: 4.4%
Core 1: 5.9%
Core 2: 5.9%
Core 3: 3.0%
Total CPU Usage: 4.8%


In [6]:
# Memory Information
print("="*40, "Memory Information", "="*40)
# get the memory details
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}")
print(f"Available: {get_size(svmem.available)}")
print(f"Used: {get_size(svmem.used)}")
print(f"Percentage: {svmem.percent}%")
print("="*20, "SWAP", "="*20)
# get the swap memory details (if exists)
swap = psutil.swap_memory()
print(f"Total: {get_size(swap.total)}")
print(f"Free: {get_size(swap.free)}")
print(f"Used: {get_size(swap.used)}")
print(f"Percentage: {swap.percent}%")

======================================== Memory Information ========================================
Total: 7.91GB
Available: 4.77GB
Used: 3.14GB
Percentage: 39.7%
==================== SWAP ====================
Total: 10.09GB
Free: 5.43GB
Used: 4.66GB
Percentage: 46.2%


In [7]:
# Disk Information
print("="*40, "Disk Information", "="*40)
print("Partitions and Usage:")
# get all disk partitions
partitions = psutil.disk_partitions()
for partition in partitions:
    print(f"=== Device: {partition.device} ===")
    print(f"  Mountpoint: {partition.mountpoint}")
    print(f"  File system type: {partition.fstype}")
    try:
        partition_usage = psutil.disk_usage(partition.mountpoint)
    except PermissionError:
        # this can be catched due to the disk that
        # isn't ready
        continue
    print(f"  Total Size: {get_size(partition_usage.total)}")
    print(f"  Used: {get_size(partition_usage.used)}")
    print(f"  Free: {get_size(partition_usage.free)}")
    print(f"  Percentage: {partition_usage.percent}%")
# get IO statistics since boot
disk_io = psutil.disk_io_counters()
print(f"Total read: {get_size(disk_io.read_bytes)}")
print(f"Total write: {get_size(disk_io.write_bytes)}")

======================================== Disk Information ========================================
Partitions and Usage:
=== Device: C:\ ===
  Mountpoint: C:\
  File system type: NTFS
  Total Size: 193.95GB
  Used: 142.24GB
  Free: 51.72GB
  Percentage: 73.3%
=== Device: D:\ ===
  Mountpoint: D:\
  File system type: 
=== Device: E:\ ===
  Mountpoint: E:\
  File system type: NTFS
  Total Size: 244.14GB
  Used: 62.00GB
  Free: 182.14GB
  Percentage: 25.4%
=== Device: F:\ ===
  Mountpoint: F:\
  File system type: NTFS
  Total Size: 290.89GB
  Used: 161.83MB
  Free: 290.73GB
  Percentage: 0.1%
Total read: 59.41GB
Total write: 33.56GB


In [8]:
# Network information
print("="*40, "Network Information", "="*40)
# get all network interfaces (virtual and physical)
if_addrs = psutil.net_if_addrs()
for interface_name, interface_addresses in if_addrs.items():
    for address in interface_addresses:
        print(f"=== Interface: {interface_name} ===")
        if str(address.family) == 'AddressFamily.AF_INET':
            print(f"  IP Address: {address.address}")
            print(f"  Netmask: {address.netmask}")
            print(f"  Broadcast IP: {address.broadcast}")
        elif str(address.family) == 'AddressFamily.AF_PACKET':
            print(f"  MAC Address: {address.address}")
            print(f"  Netmask: {address.netmask}")
            print(f"  Broadcast MAC: {address.broadcast}")
# get IO statistics since boot
net_io = psutil.net_io_counters()
print(f"Total Bytes Sent: {get_size(net_io.bytes_sent)}")
print(f"Total Bytes Received: {get_size(net_io.bytes_recv)}")

======================================== Network Information ========================================
=== Interface: Ethernet ===
=== Interface: Ethernet ===
  IP Address: 172.16.51.203
  Netmask: 255.255.255.0
  Broadcast IP: None
=== Interface: Ethernet ===
=== Interface: Ethernet 2 ===
=== Interface: Ethernet 2 ===
  IP Address: 192.168.95.1
  Netmask: 255.255.255.0
  Broadcast IP: None
=== Interface: Ethernet 2 ===
=== Interface: Ethernet 3 ===
=== Interface: Ethernet 3 ===
  IP Address: 192.168.187.1
  Netmask: 255.255.255.0
  Broadcast IP: None
=== Interface: Ethernet 3 ===
=== Interface: Loopback Pseudo-Interface 1 ===
  IP Address: 127.0.0.1
  Netmask: 255.0.0.0
  Broadcast IP: None
=== Interface: Loopback Pseudo-Interface 1 ===
Total Bytes Sent: 543.07KB
Total Bytes Received: 1.51MB


In [13]:
get_size(4294967295)

'4.00GB'

In [14]:
import sys

In [41]:
sys.call_tracing()


TypeError: call_tracing() takes exactly 2 arguments (0 given)

In [21]:
dir(sys)

['__breakpointhook__',
 '__displayhook__',
 '__doc__',
 '__excepthook__',
 '__interactivehook__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__stderr__',
 '__stdin__',
 '__stdout__',
 '_clear_type_cache',
 '_current_frames',
 '_debugmallocstats',
 '_enablelegacywindowsfsencoding',
 '_framework',
 '_getframe',
 '_git',
 '_home',
 '_xoptions',
 'api_version',
 'argv',
 'base_exec_prefix',
 'base_prefix',
 'breakpointhook',
 'builtin_module_names',
 'byteorder',
 'call_tracing',
 'callstats',
 'copyright',
 'displayhook',
 'dllhandle',
 'dont_write_bytecode',
 'exc_info',
 'excepthook',
 'exec_prefix',
 'executable',
 'exit',
 'flags',
 'float_info',
 'float_repr_style',
 'get_asyncgen_hooks',
 'get_coroutine_origin_tracking_depth',
 'get_coroutine_wrapper',
 'getallocatedblocks',
 'getcheckinterval',
 'getdefaultencoding',
 'getfilesystemencodeerrors',
 'getfilesystemencoding',
 'getprofile',
 'getrecursionlimit',
 'getrefcount',
 'getsizeof',
 'getswitchinterval',
 'gett